<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Fusion LC classifications results</font></strong></p>

<p><strong><font size="6">Processing</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [2]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile

In [3]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [4]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [5]:
run ../SRC/config.py

In [6]:
print config_parameters

{'permanent_mapset': 'PERMANENT', 'locationepsg': '31370', 'outputfolder': '../../../Results', 'gisdb': '../../GRASSDATA', 'location': 'WALOUS_31370', 'PYTHONLIB': '/usr/lib/python2.7', 'njobs': 6, 'GISBASE': '/usr/lib/grass76'}


In [7]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [8]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 8 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/grass76/bin:/usr/lib/grass76/script:/usr/lib/grass76/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 2118 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/2046,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/2046 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	
INSIDE_CAJA_PYTHON =  	
QT_ACCESSIBILITY = 1 	
LD_LIBRARY_PATH = :/usr/lib/grass76/lib 	
COMPIZ_CONFIG_PROFILE = mate 	
WINDOWPATH = 8 	
GTK_OVERLAY_SCROLLING = 0 	
PYTHONPATH

** GRASS GIS Python libraries **

In [9]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

** Other functions**

In [312]:
import compute_proportion_categorical

In [313]:
reload(compute_proportion_categorical)

<module 'compute_proportion_categorical' from '/media/tais/data/WALOUS/Processing/GithubRepository_WALOUS/SRC/compute_proportion_categorical.py'>

In [314]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function that change .csv delimiter
from ChangeCsvDelimiter import ChangeCsvDelimiter
# Import function for .gzip archive management
from gzip_management import decompress_gzip
# Import function for computation of proportion of categorical raster in zones (segments)
from compute_proportion_categorical import proportion_class_rstats
# Import function for getting list of class of a raster
from data_prep import data_prep
# Import function the allow sorting strings with number as if they was number (natural order)
from sorting_natural import natural_keys

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [250]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [251]:
def GetMapsetsAccess():
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Add mapsets with input data to the GRASS GIS research path
        gscript.run_command('g.mapsets', mapset="BINARIES,OBIA,PIXEL_ORTHO_LC,PIXEL_SENTINEL", operation="add")
        return_message = "Access to other mapset added"
    except:
        return_message += "ERROR: Add access to other Mapsets failed. Please check for problem."
    return return_message

In [252]:
def DefineComputationRegion(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        return_message = "Working on tile '%s'\n"%tile_cat
        # Extract the cutline polygon to work on
        condition = 'cat=%s'%tile_cat
        gscript.run_command('v.extract', overwrite=True, input='tiles', where=condition, output='tmp_tile')
        # Define computational region based on this polygon
        gscript.run_command('g.region', raster='segmentation') # To get resolution of segmentation 
        gscript.run_command('g.region', flags='a', vector='tmp_tile') # Keep resolution but change extent
        # Mask according to this polygon
        gscript.run_command('r.mask', overwrite=True, vector='tmp_tile')
        # Copy the mask layer
        gscript.run_command('g.copy', overwrite=True, raster='MASK,mask_copy')
        # Print
        return_message += "--> Computational region and MASK defined"
    except:
        return_message += "ERROR: Setting of computional region and MASK failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [316]:
# Function that keep only the last XX columns of a input .csv file
# TODO: Rewrite the function so that it use only Python
def CutCsv(input_csv, nb_col_from_end=0):
    import subprocess
    # Get number of colums in the .csv
    p1 = subprocess.Popen(("head -1 %s"%input_csv).split(), stdout=subprocess.PIPE)
    p2 = subprocess.Popen(("tr ',' '\\n'").split(), stdin=p1.stdout, stdout=subprocess.PIPE)
    p3 = subprocess.Popen(("wc -l").split(), stdin=p2.stdout, stdout=subprocess.PIPE)
    nb_columns = int(p3.communicate()[0])
    # Create a new .csv with only the first and the X lasts columns (X = nb_col_from_end. By default, first and last only)
    path, ext = os.path.splitext(input_csv)
    cut_csv = "%s_cut%s"%(path,ext)
    f = open(cut_csv, "w")
    subprocess.Popen(("cut -d, -f1,%s-%s %s"%(nb_columns-nb_col_from_end,nb_columns,input_csv)).split(), stdout=f)
    f.close()
    return cut_csv

In [317]:
def GetObiaAttributes(tuple_tilecat_tilezone):    
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:   
        # Define tile cat and zone of the tile
        tilecat = tuple_tilecat_tilezone[0]
        zone = tuple_tilecat_tilezone[1].upper()
        # Decompress and import attribute table of segments the current cutline
        in_path = os.path.join(data['obia_folder'],zone,'ATTRIBUTS','segs_tile%s_stats.csv.gz'%tilecat)
        decompress_path = in_path[:-3]
        decompress_gzip(in_path, decompress_path)
        if zone == 'HERVE':
            ChangeCsvDelimiter(decompress_path, '|',',') # ONLY needed for HERVE
        cut_path = CutCsv(decompress_path, nb_col_from_end=9)
        gscript.run_command('db.in.ogr', overwrite=True, input=cut_path, output='attribute')
        os.remove(decompress_path) #Remove decompressed .csv
        os.remove(cut_path)        #Remove cut .csv
        # Print
        return_message = "--> Attribute table from OBIA classification imported in GRASS"
    except:
        return_message += "ERROR: Importation of attribute table from OBIA classification failed for cutline '%s'. Please check for problem."%tilecat
    return return_message

In [318]:
def GetPixelModal():        
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Get mode of pixel-based classification for each segments
        gscript.run_command('r.mode', overwrite=True, base='segmentation', cover='ortho_lc', output='pixel_clas_mode')
        # Joint attribute from OBIA classification and modal class from pixel-based classification
        tmp_csv = tempfile.mkstemp()[1]+'.csv'
        gscript.run_command('r.univar', overwrite=True, flags='t', map='pixel_clas_mode', zones='segmentation', 
                            output=tmp_csv, separator='comma')
        gscript.run_command('db.in.ogr', overwrite=True, input=tmp_csv, output='pixel_mode')
        sql_query = "DROP TABLE IF EXISTS tmp_a"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE tmp_a AS SELECT a.*,b.min as pixel_mode FROM attribute AS a LEFT JOIN pixel_mode AS b ON a.cat_ = b.zone;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        return_message = "--> Modal class of pixel-based classification added in attributes table"
    except:
        return_message += "ERROR: Computation of modal class of pixel-based classification failed. Please check for problem."
    return  return_message

In [353]:
def GetPixelProportion():        
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        ### Compute percentage of each class from pixel-based classification
        output_csv = proportion_class_rstats("ortho_lc","segmentation",pixel_classes_list)
        ## Join to the attribute table
        table_name = '_'.join(os.path.split(output_csv)[-1].split(".csv")[0].split('_')[1:])
        gscript.run_command('db.in.ogr', overwrite=True, input=output_csv, output=table_name)
        proportion_columns = gscript.read_command('db.columns', table='prop_ortho_lc').split()[1:] #All columns except firts ('cat')
        sql_query = "DROP TABLE IF EXISTS tmp_b;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE tmp_b AS SELECT a.*,%s FROM tmp_a \
        AS a LEFT JOIN %s AS b ON a.cat_ = b.cat_;"%(','.join(['b.%s'%col for col in proportion_columns]),table_name)
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "DROP TABLE IF EXISTS tmp_a;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE tmp_a AS SELECT * FROM tmp_b;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        # Print
        return_message = "--> Proportion of pixel-based classification computed"
    except:
        return_message += "ERROR: Computation of percentage of each pixel-based class failed. Please check for problem."
    return return_message

In [224]:
def delete_tmpdata(timeclass):
    #Declare empty string that will contain the messages to return
    return_message = ''
       
    #Delete layers that are not needed anymore
    list_of_tmp_rast = ['pixel_clas','obia_clas','pixel_clas_mode',
                        'mask_copy','pixel_clas_cl_2','pixel_clas_cl_3',
                        'pixel_clas_cl_5','pixel_clas_cl_6','pixel_clas_cl_9',
                        'pixel_clas_cl_11','pixel_clas_cl_12','pixel_clas_cl_41',
                        'pixel_clas_cl_42']
    tmp_vect = 'train_points'

    try:
        gscript.run_command('g.remove', quiet=True, flags='f', type='raster', name=','.join(list_of_tmp_rast))
        gscript.run_command('g.remove', flags='f', type='vector', name=tmp_vect)
        return_message = "Temporary layers have been deleted from the mapset '%s'."%timeclass
    except:
        return_message = "ERROR: Something went wrong during the deleting of temporary layer for the mapset '%s'. Please check."%timeclass

    #Return
    return return_message

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [12]:
# Output folder stat
config_parameters['outputfolder_classfeatures'] = os.path.join(config_parameters['outputfolder'],"Classification_features")
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder_classfeatures'])

The folder '../../../Results/Classification_features' has been created


# Processing

In [191]:
# Create mapset 
launch_mapset("PERMANENT")

["Location 'WALOUS_31370' already exist",
 "'PERMANENT' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/PERMANENT'"]

**Get a list of tiles to be processed**

In [192]:
# Return cat and ZONE values of tile layer
a = gscript.parse_command('v.db.select', flags='c', map=data['tiles'][0], columns='cat,ZONE').keys()
a.sort(key=natural_keys)

In [193]:
# Create a list of tuple containing 
tile_list = [tuple(item.split("|")) for item in a]

**Get a list of classes in pixel-based classification**

In [269]:
# Define list of classes of the pixel-based classification on ORTHO
pixel_classes_list = ['11','12','2','3','41','42','5','6','9']

## Extract classification features by segment

In [194]:
#Declare empty list for saving output messages
output_message = []

In [211]:
# Define current tile
tile = tile_list[0]
print "Processing tile '%s'"%tile[0]

Processing tile '68'


In [328]:
def GetTileFeatures(current_tile):
    start_tile = start_processing() 
    #Declare empty list for saving output messages
    output_message = [] 
    #Launch mapset and create it if not exists
    message = launch_mapset(current_tile[0])  
    output_message.append(message)
    print "\n".join(message)
    #Add access to other mapset with input data
    message = GetMapsetsAccess()  
    output_message.append(message)
    print message
    #Define computational region and mask
    message = DefineComputationRegion(current_tile[0]) 
    output_message.append(message)
    print message
    #Unzip attributes from OBIA classification and import as SQlite table
    message = GetObiaAttributes(current_tile) 
    output_message.append(message)
    print message
    #Compute modal value of pixel-based classification and add to table
    message = GetPixelModal() 
    output_message.append(message)
    print message
    #Compute proportion of each class of pixel-based classification and add to table
    message = GetPixelProportion()  
    output_message.append(message)
    print message
    #Print processing time
    message = print_processing_time(start_tile, "Extraction of classification feature for tile '%s' achieved in "%current_tile[0])
    output_message.append(message)
    print message

In [354]:
GetTileFeatures(tile)

Location 'WALOUS_31370' already exist
'68' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/68'
Access to other mapset added
Working on tile '68'
--> Computational region and MASK defined
--> Attribute table from OBIA classification imported in GRASS
--> Modal class of pixel-based classification added in attributes table
--> Proportion of pixel-based classification computed
Extraction of classification feature for tile '68' achieved in 33.5 seconds
